In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
import os
# URL de connexion à la base de données Neo4j
NEO4J_URI = os.environ["GRAPH_DB_URI"]
NEO4J_USERNAME = os.environ["GRAPH_DB_USER"]
NEO4J_PASSWORD = os.environ["GRAPH_DB_PASS"]
# Clé API pour GROQ
GROQ_API_KEY = os.environ["GROQ_KEY"]


In [ ]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [ ]:
graph

In [ ]:
from langchain_groq import ChatGroq

# On récupère la clé API Groq depuis les variables d'environnement
llm = ChatGroq(
    groq_api_key=GROQ_KEY,
    model_name="llama-3.1-10b"  # un modèle plus léger pour des tests rapides
)

llm


In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [ ]:
from langchain_core.documents import Document

# Exemple de texte : biographie simplifiée de Marie Curie
texte_document = """
Marie Skłodowska Curie (née le 7 novembre 1867 à Varsovie, Pologne) 
est une physicienne et chimiste renommée pour ses travaux pionniers 
sur la radioactivité. Première femme à recevoir un prix Nobel et 
seule personne à en recevoir deux dans deux disciplines scientifiques 
différentes (physique en 1903, chimie en 1911), elle a découvert 
le polonium et le radium. En 1911, elle fonde l’Institut du Radium 
à Paris, devenu un centre majeur de recherche et de lutte contre 
le cancer.
"""

# On encapsule le texte dans un Document pour le pipeline LangChain
documents = [Document(page_content=texte_document)]



In [ ]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [ ]:
#graph_documents[0].nodes

In [ ]:
#graph_documents[0].relationships

In [ ]:
nodes = graph_documents[0].nodes

# Parcours chaque nœud et construit une requête dynamique pour chaque type de nœud
for node in nodes:
    # Utilise une f-string pour insérer dynamiquement le type de nœud
    node_query = f"""
    MERGE (n:`{node.type}` {{id: $id}})
    RETURN n
    """
    # Crée les paramètres pour le nœud courant
    node_data = {'id': node.id}

    # Exécute la requête pour chaque nœud
    graph.query(node_query, params=node_data)


In [ ]:
relationships = graph_documents[0].relationships

# Parcours chaque relation et construit une requête dynamique pour chaque type de relation
for rel in relationships:
    # Utilise une f-string pour insérer dynamiquement le type de relation
    rel_query = f"""
    MATCH (a {{id: $source_node_id}})
    MATCH (b {{id: $target_node_id}})
    MERGE (a)-[r:`{rel.type}`]->(b)
    RETURN r
    """
    # Crée les paramètres pour la relation courante
    rel_data = {
        'source_node_id': rel.source.id,
        'target_node_id': rel.target.id
    }

    # Exécute la requête pour chaque relation
    graph.query(rel_query, params=rel_data)


In [ ]:
graph.refresh_schema()

In [ ]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True)
chain

In [ ]:
# Exécute la chaîne avec une question sur Marie Curie
chain_result = chain.invoke({
    "query": "Dans quelles disciplines scientifiques Marie Curie a-t-elle reçu des prix Nobel ?"
})

# Affiche la réponse retournée par la chaîne
chain_result
